In [2]:
import requests
import csv
import os

# Authentication
app_id = "c6240510"
app_key = "119694d58027bbb42f87fe1914d67368"

# Base URL and headers
url = "https://api.schiphol.nl/public-flights/flights"
headers = {
    "Accept": "application/json",
    "app_id": app_id,
    "app_key": app_key,
    "resourceversion": "v4"
}

# Use fixed date (August 1st, 2025)
target_date = "2025-08-01"

# Output file
file_name = f"schiphol_flights_{target_date}.csv"
file_path = os.path.abspath(file_name)

# Open CSV file
with open(file_name, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Flight Number", "Airline", "Status", "Scheduled Time", "Gate", "Stand", "Terminal"])

    page = 0
    total_flights = 0

    while True:
        params = {
            "includedelays": "false",
            "sort": "+scheduleTime",
            "page": page,
            "scheduleDate": target_date
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Request failed at page {page}: {response.status_code}")
            break

        data = response.json()
        flights = data.get("flights", [])
        if not flights:
            break  # No more flights

        for f in flights:
            writer.writerow([
                f.get("flightName", ""),
                f.get("airline", {}).get("name", ""),
                ",".join(f.get("publicFlightState", {}).get("flightStates", [])),
                f.get("scheduleDateTime", ""),
                f.get("gate", ""),
                f.get("aircraftStand", ""),
                f.get("terminal", "")
            ])
        total_flights += len(flights)
        page += 1

print(f"\nDone. {total_flights} flights saved to: {file_path}")

Request failed at page 245: 204

Done. 4897 flights saved to: C:\Users\yao\schiphol_flights_2025-08-01.csv
